In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import sklearn
import sklearn.datasets
import sklearn.cross_validation
import matplotlib.pyplot as plt
import random
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Загрузим данные из файла:

In [2]:
fires = pd.read_csv('/home/denis/Downloads/forestfires.csv')

Преобразуем наши данные так, как сказано в условии. Заменим месяц на индикатор летнего времени, а столбец day теперь индикатор выходного дня. 

In [3]:
def is_weekend(day):
    if day in ('sat', 'sun'):
        return 1
    else:
        return 0

In [4]:
def is_summer(month):
    if month in ('jun', 'jul', 'aug'):
        return 1
    else:
        return 0

In [5]:
fires['day'] = list(map(lambda day : is_weekend(day), fires.day))
fires['month'] = list(map(lambda month : is_summer(month), fires.month))

In [6]:
fires.head(7)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,0,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,0,0,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,0,1,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,0,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,0,1,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0
5,8,6,1,1,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0
6,8,6,1,0,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0


Добавляем единичный столбец (свободные члены):

In [7]:
fires['w_0'] = 1

Перемешаем наши данные и разобьём их на X и y(area):

In [8]:
fires_shuffled = fires.sample(frac=1).reset_index(drop=True) 
target = fires_shuffled.area
data = fires_shuffled.drop(['area'], axis=1)

Сделаем разбиение наших данных в соотношении 7:3 (по первой части построим регрессионную модель, а ко второй применим эту модель и посчитаем среднеквадратичные ошибки):

In [9]:
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(data, target, test_size= 0.3)

Построим регрессионную модель. Создадим класс LinearRegressor. Нахождение вектора оптимальных весов $w$ может быть сделано и аналитически. Мы хотим найти такой вектор весов $w$, чтобы вектор $y$, соответствующий целевому признаку, получался умножением матрицы $X$ (состоящей из всех признаков объектов обучающей выборки, кроме целевого) на вектор весов $w$. То есть, чтобы выполнялось матричное уравнение: $$y = Xw$$ Решением этого уравнения будет: $$w = (X^TX)^{-1}X^Ty$$ Решать матричные уравнения мы можем с помощью функции linalg.solve из библиотеки numpy.

In [10]:
class LinearRegressor():
    weights = []
    def __init__(self):
        self.weights = []
    def fit(self, X, y):
        self.weights = np.linalg.solve(np.dot(np.transpose(X), X), np.dot(np.transpose(X), y))
    def predict(self, X):
        return np.dot(X, self.weights)

Построим регрессионную модель и предскажем area. Посмотрим на Веса и среднеквадратичную ошибку

In [13]:
first_model = LinearRegressor()
first_model.fit(X_train.as_matrix(), y_train.as_matrix())
predictions = first_model.predict(X_test.as_matrix())

In [14]:
first_model.weights

array([  2.35861032e+00,   1.77333175e+00,  -1.21712281e+01,
         6.13556947e+00,  -4.96481464e-02,   1.18969111e-01,
        -1.49195462e-02,  -5.43525543e-01,   1.65427298e+00,
        -1.23370546e-01,   2.22993642e+00,  -1.73312859e+01,
        -3.18819901e+01])

Видим, что ошибки получились большими. Значит, скорее всего, такая модель нам не подходит.